File này để chuẩn bị dữ liệu, chia train là 2020, test là 2021

In [1]:
import pandas as pd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle
rs = 12

In [2]:
df = pd.read_excel('../../data/DataFile.xlsx')
df['DateKey'] = df['DateKey'].map(lambda x: f'{str(x)[:4]}-{str(x)[4:6]}-{str(x)[6:]}')
df['DateKey'] = pd.to_datetime(df['DateKey'])

In [3]:
# chỉ lấy 2020
df = df[df['DateKey'].dt.year == 2020]
df.to_csv('../../data/data.csv', index=False)
df.shape

(19804, 7)

# Xử lý dữ liệu

In [4]:
%run ../../processing_data.py ../../data/data.csv ../../data/data_processed.csv

# Tạo RFM data

In [5]:
data_processed = pd.read_csv('../../data/data_processed.csv', parse_dates=[0])

In [6]:
data_processed.head()

,DateKey,User number,Full name,Product Name,Product Type,Quantity,Regular price
0,2020-12-31,101010971,Nguyễn Mai,Membership_3M,Membership,1,837000
1,2020-10-09,105001744,Bui Thi Thanh Tuyền,Fitto Superfit,ProductGood,1,35000
2,2020-11-30,101009401,Dat Luong,Membership_6M,Membership,1,1600000
3,2020-12-14,101001673,Lưu Mộc Lợi,Membership_2M,Membership,1,558000
4,2020-12-16,101011036,Nguyen Nhu Quynh,Membership_12M,Membership,1,2800000


In [7]:
data_processed['Product Name'].describe()

count             19215
unique               32
top       Membership_1M
freq               7697
Name: Product Name, dtype: object

In [8]:

recency_unit = 1
snapshot_date = dt.datetime(year=2021, month=12, day=31)

def get_recency(d):
    return (snapshot_date - d.max()).days // recency_unit

# recency: lấy ngày snapshot - ngày mua cuối
# frequency: tổng quantity
# monetary: số tiền mua

scaler = MinMaxScaler()

def get_rfm_data(dataframe):
    dataframe = (dataframe
            .groupby(['User number'])
            .agg({'DateKey': get_recency, 'Quantity': 'count', 'Regular price': 'sum'})
    )
    user_id = dataframe.index
    rfm = dataframe.values
    rfm = scaler.fit_transform(rfm) # normalize
    return pd.DataFrame(rfm, index=user_id, columns=['Recency', 'Frequency', 'Moneytary'])

In [9]:
rfm_data = get_rfm_data(data_processed)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f, protocol=pickle.HIGHEST_PROTOCOL)


rfm_data.to_csv('../../data/rfm_data.csv')
rfm_data.shape

(8806, 3)